In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Saver for rays + reproducible projections (yt + manual) for a TNG cutout,
using observed M61 geometry from a CSV and galaxy orientation saved into the cutout.

Outputs (in OUTROOT/):
  - rays_sid<SID>.csv                 endpoints, anchor, LOS & "north" (ABSOLUTE, ckpc/h) for α grid, both modes
  - orient_peralpha_sid<SID>.csv     per-α camera vectors (native) for both modes
  - orient_header_sid<SID>.json      constants, base matrices, units, conventions
  - debug_manual_alpha###_<mode>.png manual panel for one sampled α (noflip/flip)
  - debug_yt_alpha###_<mode>.png     yt off-axis projection for same α (noflip/flip)
  - compare_alpha###_<mode>.png      side-by-side comparison

Conventions:
  - Saved coordinates for rays are in code_length = ckpc/h.
  - OBS→native mapping: v_nat = v_obs @ R_cur
  - native→OBS mapping: X_obs = X_nat @ R_cur.T
  - Base matrices are OBS→native.

This script:
  1) Reads observed M61 geometry from M61_DIISC_Table1_Table2.csv
  2) Reads orientation from cutout /Orientation (v3_hat, etc.)
     Falls back to recomputing v3_hat from stars if /Orientation is missing.
  3) Builds R_base matrices for both disk-normal choices (noflip/flip)
  4) Sweeps α and saves ray endpoints + camera vectors
  5) Produces debug comparison images for one reproducible SAMPLE_ALPHA
"""

import os, sys, math, glob, json, traceback, time
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import h5py
import yt
import trident

# =========================
# USER CONFIG — PATHS
# =========================
CUTOUT_H5 = r"/Users/wavefunction/ASU Dropbox/Tanmay Singh/M61/data/cutout_398784.hdf5"

# Group catalog base (contains groups_099/)
GROUPCAT_BASE = r"/Users/wavefunction/ASU Dropbox/Tanmay Singh/COS_GASS"
SNAP = 99
SID  = 398784

# Observed M61 table file (contains inclination, PA, theta, impact, Rvir, etc.)
M61_TABLE_CSV = r"/Users/wavefunction/ASU Dropbox/Tanmay Singh/M61/data/M61_DIISC_Table1_Table2.csv"
TARGET_GALAXY_NAME = "M61/NGC4303"   # row selector

# Alpha sweep
ALL_ALPHAS = list(range(0, 360, 1))
SAMPLE_ALPHA = int(np.random.default_rng(SID).integers(0, 360))

# Plots only (saved data stays in ckpc/h)
FOV_KPC   = 300.0
WIDTH_KPC = 2.0 * FOV_KPC
DEPTH_KPC = WIDTH_KPC

# Optional PCA aperture for stars if /Orientation is missing
USE_RHALF_APERTURE = True
RHALF_MULTIPLIER   = 10.0

# Manual panel: use gas for the manual density map if present; else stars
MANUAL_USE_GAS_IF_AVAILABLE = True

# Output root
OUTROOT = rf"/Users/wavefunction/ASU Dropbox/Tanmay Singh/M61/data/rays_and_recipes_sid{SID}_snap{SNAP:03d}"
os.makedirs(OUTROOT, exist_ok=True)

mpl.rcParams.update({
    "figure.dpi": 140, "savefig.dpi": 240,
    "figure.facecolor": "#0f111a", "axes.facecolor": "#0f111a", "savefig.facecolor": "#0f111a",
    "axes.edgecolor": "#eaeaea", "axes.labelcolor": "#eaeaea",
    "xtick.color": "#eaeaea", "ytick.color": "#eaeaea",
    "text.color": "#eaeaea", "axes.grid": False, "font.size": 11,
})

# ========= math / PCA / PBC =========
def unit(v):
    v = np.asarray(v, float)
    n = np.linalg.norm(v)
    return v / n if n > 0 else v

def rodrigues_axis_angle(axis, theta):
    a = unit(axis); ax, ay, az = a
    c, s = math.cos(theta), math.sin(theta); C = 1 - c
    return np.array([
        [c+ax*ax*C,   ax*ay*C-az*s, ax*az*C+ay*s],
        [ay*ax*C+az*s, c+ay*ay*C,   ay*az*C-ax*s],
        [az*ax*C-ay*s, az*ay*C+ax*s, c+az*az*C  ]
    ], float)

def R_from_u_to_v(u, v):
    u, v = unit(u), unit(v)
    c = float(np.clip(np.dot(u, v), -1.0, 1.0))
    if c > 1 - 1e-12:
        return np.eye(3)
    if c < -1 + 1e-12:
        a = unit(np.cross(u, [1,0,0]) if abs(u[0]) < 0.9 else np.cross(u, [0,1,0]))
        return rodrigues_axis_angle(a, math.pi)
    axis = unit(np.cross(u, v))
    return rodrigues_axis_angle(axis, math.acos(c))

def rot_x(theta_deg):
    t = np.radians(theta_deg); c, s = np.cos(t), np.sin(t)
    return np.array([[1,0,0],[0,c,-s],[0,s,c]], float)

def minimal_image_delta(dpos, box):
    return (dpos + 0.5*box) % box - 0.5*box

def recenter_positions(x_ckpch, center_ckpch, box_ckpch):
    return minimal_image_delta(x_ckpch - center_ckpch[None,:], box_ckpch)

def pca3_weighted(X, w):
    X = np.asarray(X, float); w = np.asarray(w, float)
    wsum = np.sum(w)
    xc = np.sum(X*w[:,None], axis=0) / max(wsum, 1e-30)
    X0 = X - xc
    C = (X0*w[:,None]).T @ X0 / max(wsum, 1e-30)
    evals, evecs = np.linalg.eigh(C)
    idx = np.argsort(evals)[::-1]
    return evals[idx], evecs[:, idx], xc

# ========= groupcat =========
def _groupcat_dir(base, snap):
    # allow base already pointing to groups_099
    if os.path.basename(base) == f"groups_{snap:03d}" and os.path.isdir(base):
        return base
    d = os.path.join(base, f"groups_{snap:03d}")
    return d if os.path.isdir(d) else base

def _chunk_path(base, snap, filenum):
    d = _groupcat_dir(base, snap)
    return os.path.join(d, f"fof_subhalo_tab_{snap:03d}.{filenum}.hdf5")

def _find_chunks(base, snap):
    d = _groupcat_dir(base, snap)
    files = sorted(
        glob.glob(os.path.join(d, f"fof_subhalo_tab_{snap:03d}.*.hdf5")),
        key=lambda p: int(os.path.splitext(p)[0].split(".")[-1]) if os.path.splitext(p)[0].split(".")[-1].isdigit() else -1
    )
    alt = os.path.join(base, f"groups_{snap:03d}.hdf5")
    if not files and os.path.exists(alt):
        files = [alt]
    return files

def _read_header_any(base, snap):
    p0 = _chunk_path(base, snap, 0)
    candidates = [p0] if os.path.exists(p0) else _find_chunks(base, snap)
    if not candidates:
        raise FileNotFoundError(f"No group catalog files under {base} for snap {snap}.")
    for ff in candidates:
        try:
            with h5py.File(ff, "r") as f:
                h  = float(f["Header"].attrs["HubbleParam"])
                bs = float(f["Header"].attrs["BoxSize"])
                offs = None
                for key in ("FileOffsets_Subhalo", "FileOffsets_SubFindSubhalo", "FileOffsets_SubFind"):
                    if key in f["Header"].attrs:
                        offs = np.array(f["Header"].attrs[key], dtype=np.int64)
                        break
                return h, bs, offs
        except Exception:
            continue
    raise RuntimeError("Unable to read header from any chunk.")

def _is_valid_subhalo_chunk(path):
    try:
        with h5py.File(path, "r") as f:
            return ("Subhalo" in f and "SubhaloPos" in f["Subhalo"]
                    and f["Subhalo"]["SubhaloPos"].shape[0] > 0)
    except Exception:
        return False

def read_single_subhalo(base, snap, subhalo_id):
    subhalo_id = int(subhalo_id)
    h, boxsize, offsets = _read_header_any(base, snap)

    if offsets is not None:
        rel = subhalo_id - offsets
        fileNum = int(np.max(np.where(rel >= 0)))
        local_index = int(rel[fileNum])
        candidate = _chunk_path(base, snap, fileNum)
        if not os.path.exists(candidate):
            raise FileNotFoundError(f"Missing expected chunk {fileNum}: {candidate}")
        if not _is_valid_subhalo_chunk(candidate):
            raise RuntimeError(f"Chunk {os.path.basename(candidate)} lacks Subhalo datasets (cloud stub?).")
        with h5py.File(candidate, "r") as f:
            sub = f["Subhalo"]
            pos  = np.asarray(sub["SubhaloPos"][local_index], dtype=np.float64)
            hmr  = np.asarray(sub["SubhaloHalfmassRadType"][local_index], dtype=np.float64)
            spin = np.asarray(sub["SubhaloSpin"][local_index], dtype=np.float64)
        return {"h": h, "BoxSize": boxsize,
                "SubhaloPos": pos, "SubhaloHalfmassRadType": hmr, "SubhaloSpin": spin}

    files = _find_chunks(base, snap)
    remaining = subhalo_id
    for ff in files:
        if not _is_valid_subhalo_chunk(ff):
            continue
        with h5py.File(ff, "r") as f:
            n_here = f["Subhalo"]["SubhaloPos"].shape[0]
            if remaining >= n_here:
                remaining -= n_here
                continue
            pos  = np.asarray(f["Subhalo"]["SubhaloPos"][remaining], dtype=np.float64)
            hmr  = np.asarray(f["Subhalo"]["SubhaloHalfmassRadType"][remaining], dtype=np.float64)
            spin = np.asarray(f["Subhalo"]["SubhaloSpin"][remaining], dtype=np.float64)
        return {"h": h, "BoxSize": boxsize,
                "SubhaloPos": pos, "SubhaloHalfmassRadType": hmr, "SubhaloSpin": spin}

    raise IndexError(f"SubhaloID {subhalo_id} not found in usable chunks.")

# ========= cutout =========
def read_cutout_stars_gas(h5_path):
    with h5py.File(h5_path, "r") as f:
        Xs = np.asarray(f["/PartType4/Coordinates"][...], dtype=np.float64)
        Ms = np.asarray(f["/PartType4/Masses"][...],      dtype=np.float64)
        # velocities not needed here
        if "/PartType0/Coordinates" in f and "/PartType0/Masses" in f:
            Xg = np.asarray(f["/PartType0/Coordinates"][...], dtype=np.float64)
            Mg = np.asarray(f["/PartType0/Masses"][...],      dtype=np.float64)
        else:
            Xg, Mg = None, None
    return Xs, Ms, Xg, Mg

def read_orientation_from_cutout(h5_path):
    """
    Returns v3_hat_native (unit vector) if /Orientation exists, else None.
    """
    try:
        with h5py.File(h5_path, "r") as f:
            if "Orientation" not in f:
                return None
            g = f["Orientation"]
            if "v3_hat" in g:
                v3 = np.asarray(g["v3_hat"][...], dtype=np.float64)
                return unit(v3)
    except Exception:
        return None
    return None

# ========= observed geometry =========
def load_observed_geometry(csv_path, galaxy_name):
    df = pd.read_csv(csv_path)
    # robust matching (exact or contains)
    m = (df["galaxy_name"].astype(str) == galaxy_name)
    if not np.any(m):
        m = df["galaxy_name"].astype(str).str.contains(galaxy_name, regex=False, na=False)
    if not np.any(m):
        raise RuntimeError(f"Could not find galaxy_name='{galaxy_name}' in {csv_path}")
    row = df.loc[m].iloc[0]

    INC_DEG   = float(row["inclination_deg"])
    RHO_KPC   = float(row["impact_kpc"])
    PHI_DEG   = float(row["theta_deg"])           # this is your in-plane azimuth choice
    R_VIR_KPC = float(row["Rvir_kpc"])
    PA_DEG    = float(row["PA_deg"])              # not used below unless you decide to include it
    B_OVER_A  = float(row["b_over_a"])            # not used below unless you decide to include it

    return dict(
        INC_DEG=INC_DEG,
        RHO_KPC=RHO_KPC,
        PHI_DEG=PHI_DEG,
        R_VIR_KPC=R_VIR_KPC,
        PA_DEG=PA_DEG,
        B_OVER_A=B_OVER_A,
        row=row.to_dict()
    )

# ========= core geometry builders =========
def build_R_bases_from_v3_hat(v3_hat_native, inc_deg):
    """
    Build base OBS→native matrices for both normal choices.
    Steps:
      1) Rotate so disk normal maps to +z_obs  (R_face: native→obs if applied as X_obs = X_nat @ R_face.T)
         Here we need OBS→native. For consistency with existing code we build R_face as OBS→native mapping:
           v_nat = v_obs @ R_face
         That means we want (disk normal in native) to map to +z in OBS when going native→obs.
         Using R_from_u_to_v(u, v) gives active rotation mapping u->v in the same coordinate system.
         In our convention, we use OBS→native as a right-multiply for row vectors.
    We keep your original convention exactly:
      R_face = R_from_u_to_v(v3_hat, +z_obs)   (OBS→native)
      R_base = rot_x(inc) @ R_face
    """
    v3_hat_native = unit(v3_hat_native)

    R_face_noflip = R_from_u_to_v(v3_hat_native, np.array([0,0,1.0]))
    R_base_noflip = rot_x(inc_deg) @ R_face_noflip

    R_face_flip = R_from_u_to_v(-v3_hat_native, np.array([0,0,1.0]))
    R_base_flip = rot_x(inc_deg) @ R_face_flip

    return R_base_noflip, R_base_flip, v3_hat_native, -v3_hat_native

def sightline_endpoints_codeunits(center_ckpch, R_cur, rho_ckpch, phi_deg, half_len_ckpch):
    """
    Build endpoints directly in code_length (ckpc/h).
    OBS→native mapping uses @ R_cur.
    """
    x = rho_ckpch * math.cos(math.radians(phi_deg))
    y = rho_ckpch * math.sin(math.radians(phi_deg))
    r_obs = np.array([x, y, 0.0], float)
    ez_obs = np.array([0.0, 0.0, 1.0])

    r_nat = r_obs @ R_cur            # OBS→native
    L_nat = unit(ez_obs @ R_cur)     # native LOS

    p0 = center_ckpch + r_nat - half_len_ckpch * L_nat
    p1 = center_ckpch + r_nat + half_len_ckpch * L_nat
    anchor = center_ckpch + r_nat
    return p0, p1, anchor, L_nat

def manual_panel(X_map_rel_ckpch, W_map, R_cur, h, center_ckpch,
                 p0_ckpch, p1_ckpch, alpha_deg, mode, out_png,
                 inc_deg, rho_kpc, phi_deg, rhalf_star_ckpch,
                 fov_kpc=300.0, nbin=700, title_prefix="Manual"):
    """
    Manual projection to OBS plane using native→OBS: X_obs = X_nat @ R_cur.T.
    """
    Xobs_kpc = (X_map_rel_ckpch / h) @ R_cur.T
    R = float(fov_kpc)

    H, _, _ = np.histogram2d(
        Xobs_kpc[:,0], Xobs_kpc[:,1], bins=nbin,
        range=[[-R,R],[-R,R]], weights=W_map
    )

    fig, ax = plt.subplots(1, 1, figsize=(6.2, 5.8))
    im = ax.imshow(np.log10(H.T + 1e-12), origin="lower",
                   extent=[-R, R, -R, R], cmap="magma", vmin=2.0, vmax=8.0)
    cb = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.03)
    cb.set_label("log10 projected mass [arb]")

    # disk ellipse proxy using stellar rhalf (kpc)
    a = 3.0 * (rhalf_star_ckpch / h)
    b = a * math.cos(math.radians(inc_deg))
    ax.add_patch(Ellipse((0,0), 2*a, 2*b, edgecolor="#00d1b2", facecolor="none", lw=2.0, zorder=6))

    # sightline point in OBS plane
    x_sl = rho_kpc * math.cos(math.radians(phi_deg))
    y_sl = rho_kpc * math.sin(math.radians(phi_deg))
    ax.scatter(x_sl, y_sl, s=150, marker="*", c="#00e5ff", edgecolor="#ff3d57",
               linewidths=0.9, zorder=8)

    # LOS segment (in OBS plane, should collapse to a point)
    p0_rel_obs_kpc = ((p0_ckpch - center_ckpch) / h) @ R_cur.T
    p1_rel_obs_kpc = ((p1_ckpch - center_ckpch) / h) @ R_cur.T
    ax.plot([p0_rel_obs_kpc[0], p1_rel_obs_kpc[0]],
            [p0_rel_obs_kpc[1], p1_rel_obs_kpc[1]],
            color="#ff5f78", lw=2.0, alpha=0.9, zorder=8)

    dx, dy = p1_rel_obs_kpc[0]-p0_rel_obs_kpc[0], p1_rel_obs_kpc[1]-p0_rel_obs_kpc[1]
    ax.text(-R+10, -R+14, f"Δx={dx:.3e}, Δy={dy:.3e} kpc", color="#ddd", fontsize=9)

    ax.set_aspect("equal")
    ax.set_xlim(-R, R); ax.set_ylim(-R, R)
    ax.set_xlabel("x_obs [kpc]"); ax.set_ylabel("y_obs [kpc]")
    ax.set_title(f"{title_prefix}  α={alpha_deg:03d}° — width={int(2*R)} kpc | {mode}")

    plt.savefig(out_png, bbox_inches="tight")
    plt.close(fig)

# ========= main =========
def main():
    # 0) Observed geometry for M61
    obs = load_observed_geometry(M61_TABLE_CSV, TARGET_GALAXY_NAME)
    INC_DEG   = obs["INC_DEG"]
    RHO_KPC   = obs["RHO_KPC"]
    PHI_DEG   = obs["PHI_DEG"]
    R_VIR_KPC = obs["R_VIR_KPC"]

    # 1) Load groupcat (h, center, rhalf)
    sh = read_single_subhalo(GROUPCAT_BASE, SNAP, SID)
    h            = float(sh["h"])
    box_ckpch    = float(sh["BoxSize"])
    center_ckpch = np.array(sh["SubhaloPos"], float)
    rhalf_star_ckpch = float(sh["SubhaloHalfmassRadType"][4])

    print(f"[OK] Groupcat: h={h:.6f}, BoxSize={box_ckpch:.1f} ckpc/h")
    print(f"[OK] Center: {center_ckpch} ckpc/h | rhalf*(stars)={rhalf_star_ckpch:.3f} ckpc/h")

    # 2) Read cutout particles (for fallback PCA and for manual panel map)
    Xs_ckpch, Ms_1e10, Xg_ckpch, Mg_1e10 = read_cutout_stars_gas(CUTOUT_H5)
    have_gas = (Xg_ckpch is not None and Mg_1e10 is not None)

    # Recenter around catalog center (PBC-aware)
    Xs_rel_ckpch = recenter_positions(Xs_ckpch, center_ckpch, box_ckpch)

    if MANUAL_USE_GAS_IF_AVAILABLE and have_gas:
        Xg_rel_ckpch = recenter_positions(Xg_ckpch, center_ckpch, box_ckpch)
        X_map_rel_ckpch = Xg_rel_ckpch
        W_map = Mg_1e10
        manual_title = "Manual (gas)"
    else:
        X_map_rel_ckpch = Xs_rel_ckpch
        W_map = Ms_1e10
        manual_title = "Manual (stars)"

    # 3) Get disk normal v3_hat from cutout /Orientation if available
    v3_hat_native = read_orientation_from_cutout(CUTOUT_H5)

    if v3_hat_native is None:
        # fallback: recompute PCA v3_hat from stars (optionally in an aperture)
        if USE_RHALF_APERTURE and rhalf_star_ckpch > 0:
            R = np.linalg.norm(Xs_rel_ckpch, axis=1)
            keep = (R <= RHALF_MULTIPLIER * rhalf_star_ckpch)
            Xp = Xs_rel_ckpch[keep]
            Wp = Ms_1e10[keep]
            print(f"[WARN] /Orientation missing; PCA fallback using stars within {RHALF_MULTIPLIER:.1f} rhalf: kept {keep.sum()} stars")
        else:
            Xp = Xs_rel_ckpch
            Wp = Ms_1e10
            print("[WARN] /Orientation missing; PCA fallback using all stars in cutout")

        _, evecs3, _ = pca3_weighted(Xp, Wp)
        v3_hat_native = unit(evecs3[:,2])

    # 4) Build base matrices (OBS→native), noflip/flip
    R_base_noflip, R_base_flip, v3_hat, v3_hat_flip = build_R_bases_from_v3_hat(v3_hat_native, INC_DEG)

    # 5) Build per-α tables (saved in ckpc/h)
    rho_ckpch      = RHO_KPC * h
    half_len_ckpch = (1.5 * R_VIR_KPC) * h

    ey_obs = np.array([0.0, 1.0, 0.0])
    ez_obs = np.array([0.0, 0.0, 1.0])

    rows = []
    rows_orient = []

    for alpha in ALL_ALPHAS:
        for mode, R_base, axis in [
            ("noflip", R_base_noflip, v3_hat),
            ("flip",   R_base_flip,   v3_hat_flip),
        ]:
            S_alpha = rodrigues_axis_angle(axis, math.radians(alpha))
            R_cur   = R_base @ S_alpha                     # OBS→native

            # Camera basis (native) for yt/off-axis reproducibility
            los_nat   = unit(ez_obs @ R_cur)
            north_nat = unit(ey_obs @ R_cur)

            p0, p1, anchor, L_nat = sightline_endpoints_codeunits(
                center_ckpch, R_cur, rho_ckpch, PHI_DEG, half_len_ckpch
            )

            rows.append({
                "SubhaloID": SID,
                "alpha_deg": alpha,
                "mode": mode,
                "inc_deg": INC_DEG,
                "rho_kpc": RHO_KPC,
                "phi_deg": PHI_DEG,
                "Rvir_kpc": R_VIR_KPC,
                "p0_X_ckpch_abs": p0[0], "p0_Y_ckpch_abs": p0[1], "p0_Z_ckpch_abs": p0[2],
                "p1_X_ckpch_abs": p1[0], "p1_Y_ckpch_abs": p1[1], "p1_Z_ckpch_abs": p1[2],
                "anchor_X_ckpch_abs": anchor[0], "anchor_Y_ckpch_abs": anchor[1], "anchor_Z_ckpch_abs": anchor[2],
                "los_x":  los_nat[0], "los_y":  los_nat[1], "los_z":  los_nat[2],
                "north_x": north_nat[0], "north_y": north_nat[1], "north_z": north_nat[2],
            })

            rows_orient.append({
                "alpha_deg": alpha,
                "mode": mode,
                "los_x": los_nat[0], "los_y": los_nat[1], "los_z": los_nat[2],
                "north_x": north_nat[0], "north_y": north_nat[1], "north_z": north_nat[2],
            })

    rays_csv = os.path.join(OUTROOT, f"rays_sid{SID}.csv")
    pd.DataFrame(rows).to_csv(rays_csv, index=False)

    orient_csv = os.path.join(OUTROOT, f"orient_peralpha_sid{SID}.csv")
    pd.DataFrame(rows_orient).to_csv(orient_csv, index=False)

    header_json = os.path.join(OUTROOT, f"orient_header_sid{SID}.json")
    header = {
        "SID": SID, "SNAP": SNAP,
        "cutout_h5": CUTOUT_H5,
        "groupcat_base": GROUPCAT_BASE,
        "h": h,
        "BoxSize_ckpc_h": box_ckpch,
        "center_ckpc_h": center_ckpch.tolist(),
        "rhalf_star_ckpc_h": rhalf_star_ckpch,
        "observed_row": obs["row"],
        "used_geometry": {
            "INC_DEG": INC_DEG,
            "RHO_KPC": RHO_KPC,
            "PHI_DEG": PHI_DEG,
            "R_VIR_KPC": R_VIR_KPC
        },
        "bases_obs_to_native": {
            "R_base_noflip": R_base_noflip.tolist(),
            "R_base_flip": R_base_flip.tolist(),
            "v3_hat_native": v3_hat.tolist(),
            "v3_hat_native_flip": v3_hat_flip.tolist(),
        },
        "conventions": {
            "native_to_obs": "X_obs = X_nat @ R_cur.T",
            "obs_to_native": "v_nat = v_obs @ R_cur",
            "los_nat": "los_nat = unit(ez_obs @ R_cur)",
            "north_nat": "north_nat = unit(ey_obs @ R_cur)",
            "alpha_rotation": "R_cur = R_base(mode) @ Rodrigues(axis=v3_hat_mode, theta=alpha)",
        },
        "units": {
            "positions_saved": "ckpc/h (code_length)",
            "vectors_saved": "dimensionless unit vectors in native coords",
            "rho_input": "kpc",
            "virial_radius_input": "kpc",
        }
    }
    with open(header_json, "w") as f:
        json.dump(header, f, indent=2)

    print(f"[OK] wrote rays CSV: {rays_csv}")
    print(f"[OK] wrote per-α orientation CSV: {orient_csv}")
    print(f"[OK] wrote header JSON: {header_json}")

    # ========== Debug: manual vs yt for one reproducible alpha ==========
    ds = yt.load(CUTOUT_H5)
    trident.add_ion_fields(ds, ions=["H I"])

    center_abs = ds.arr(center_ckpch, "code_length")
    width_tuple = (WIDTH_KPC, "kpc")
    depth_tuple = (DEPTH_KPC, "kpc")

    for mode, R_base, axis in [
        ("noflip", R_base_noflip, v3_hat),
        ("flip",   R_base_flip,   v3_hat_flip),
    ]:
        alpha = SAMPLE_ALPHA
        S_alpha = rodrigues_axis_angle(axis, math.radians(alpha))
        R_cur   = R_base @ S_alpha

        p0, p1, anchor, los_nat = sightline_endpoints_codeunits(
            center_ckpch, R_cur, rho_ckpch, PHI_DEG, half_len_ckpch
        )
        north_nat = unit(np.array([0.0, 1.0, 0.0]) @ R_cur)

        man_png = os.path.join(OUTROOT, f"debug_manual_alpha{alpha:03d}_{mode}.png")
        manual_panel(
            X_map_rel_ckpch=X_map_rel_ckpch,
            W_map=W_map,
            R_cur=R_cur,
            h=h,
            center_ckpch=center_ckpch,
            p0_ckpch=p0,
            p1_ckpch=p1,
            alpha_deg=alpha,
            mode=mode,
            out_png=man_png,
            inc_deg=INC_DEG,
            rho_kpc=RHO_KPC,
            phi_deg=PHI_DEG,
            rhalf_star_ckpch=rhalf_star_ckpch,
            fov_kpc=FOV_KPC,
            nbin=700,
            title_prefix=manual_title
        )

        sp = ds.arr(p0, "code_length")
        ep = ds.arr(p1, "code_length")
        mp = ds.arr(anchor, "code_length")

        pw = yt.OffAxisProjectionPlot(
            ds,
            normal=los_nat,
            fields=("gas", "H_p0_number_density"),
            center=center_abs,
            width=width_tuple,
            depth=depth_tuple,
            north_vector=north_nat,
            weight_field=None,
        )
        pw.set_cmap(("gas", "H_p0_number_density"), "viridis")
        pw.annotate_line(sp, ep, coord_system="data", color="deeppink", linewidth=2.0)
        pw.annotate_marker(mp, coord_system="data", marker="*", s=160, color="yellow")
        pw.annotate_title(
            "yt Off-axis projection — H I number density\n"
            f"SID={SID}  α={alpha:03d}°  mode={mode} | inc={INC_DEG:.2f}°  ρ={RHO_KPC:.1f} kpc  φ={PHI_DEG:.1f}°"
        )

        yt_base = os.path.join(OUTROOT, f"debug_yt_alpha{alpha:03d}_{mode}")
        saved = pw.save(yt_base)  # returns list of filenames
        yt_png = saved[0] if isinstance(saved, (list, tuple)) and len(saved) > 0 else (yt_base + ".png")

        # side-by-side
        fig, axes = plt.subplots(1, 2, figsize=(12.8, 5.6))
        for ax, path, title in [
            (axes[0], man_png, manual_title),
            (axes[1], yt_png,  "yt OffAxisProjectionPlot"),
        ]:
            img = plt.imread(path)
            ax.imshow(img)
            ax.set_title(title)
            ax.axis("off")

        fig.suptitle(f"Subhalo {SID} | α={alpha:03d}° | mode={mode}", y=0.98)
        cmp_png = os.path.join(OUTROOT, f"compare_alpha{alpha:03d}_{mode}.png")
        plt.tight_layout()
        plt.savefig(cmp_png, dpi=200)
        plt.close(fig)

        print(f"[OK] wrote {man_png}")
        print(f"[OK] wrote {yt_png}")
        print(f"[OK] wrote {cmp_png}")

    print("[DONE]", OUTROOT)

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print("[FATAL]", e)
        traceback.print_exc()
        sys.exit(1)


No ion table data file found in /Users/wavefunction/github_repos/COS-GASS/notebooks/y
The following data files are available:
1) hm2012_ss_hr.h5.gz   329 MB   Haardt Madau 2012 Background High Res w/ Self-Shielding
2) hm2012_hr.h5.gz   296 MB   Haardt Madau 2012 Background High Res (Preferred)
3) hm2012_lr.h5.gz    42 MB   Haardt Madau 2012 Background Low Res 
4) fg2009_ss_hr.h5.gz   264 MB   Faucher-Giguere 2009 Background High Res w/ Self-Shielding
5) fg2009_hr.h5.gz   262 MB   Faucher-Giguere 2009 Background High Res 
6) fg2009_lr.h5.gz    37 MB   Faucher-Giguere 2009 Background Low Res 

Which number would you like to download and use? [2]



  Unzipping file: hm2012_ss_hr.h5.gz
[OK] Groupcat: h=0.677400, BoxSize=35000.0 ckpc/h
[OK] Center: [14431.12304688  8202.77539062   834.62689209] ckpc/h | rhalf*(stars)=7.261 ckpc/h
[OK] wrote rays CSV: /Users/wavefunction/ASU Dropbox/Tanmay Singh/M61/data/rays_and_recipes_sid398784_snap099/rays_sid398784.csv
[OK] wrote per-α orientation CSV: /Users/wavefunction/ASU Dropbox/Tanmay Singh/M61/data/rays_and_recipes_sid398784_snap099/orient_peralpha_sid398784.csv
[OK] wrote header JSON: /Users/wavefunction/ASU Dropbox/Tanmay Singh/M61/data/rays_and_recipes_sid398784_snap099/orient_header_sid398784.json


yt : [INFO     ] 2026-02-03 12:44:58,432 Calculating time from 1.000e+00 to be 4.356e+17 seconds
yt : [INFO     ] 2026-02-03 12:44:58,478 Parameters: current_time              = 4.355810528213311e+17 s
yt : [INFO     ] 2026-02-03 12:44:58,478 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2026-02-03 12:44:58,479 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2026-02-03 12:44:58,480 Parameters: domain_right_edge         = [35000. 35000. 35000.]
yt : [INFO     ] 2026-02-03 12:44:58,480 Parameters: cosmological_simulation   = True
yt : [INFO     ] 2026-02-03 12:44:58,480 Parameters: current_redshift          = 2.220446049250313e-16
yt : [INFO     ] 2026-02-03 12:44:58,480 Parameters: omega_lambda              = 0.6911
yt : [INFO     ] 2026-02-03 12:44:58,481 Parameters: omega_matter              = 0.3089
yt : [INFO     ] 2026-02-03 12:44:58,481 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2026-02-03 12:44:58,482 Parameters: hubble_con

[OK] wrote /Users/wavefunction/ASU Dropbox/Tanmay Singh/M61/data/rays_and_recipes_sid398784_snap099/debug_manual_alpha109_noflip.png
[OK] wrote /Users/wavefunction/ASU Dropbox/Tanmay Singh/M61/data/rays_and_recipes_sid398784_snap099/debug_yt_alpha109_noflip_OffAxisProjection_H_p0_number_density.png
[OK] wrote /Users/wavefunction/ASU Dropbox/Tanmay Singh/M61/data/rays_and_recipes_sid398784_snap099/compare_alpha109_noflip.png


yt : [INFO     ] 2026-02-03 12:45:07,532 xlim = -203.219972 203.219972
yt : [INFO     ] 2026-02-03 12:45:07,532 ylim = -203.219972 203.219972
yt : [INFO     ] 2026-02-03 12:45:07,533 zlim = -203.219972 203.219972
yt : [INFO     ] 2026-02-03 12:45:07,533 Making a fixed resolution buffer of (('gas', 'H_p0_number_density')) 800 by 800
yt : [INFO     ] 2026-02-03 12:45:10,528 Saving plot /Users/wavefunction/ASU Dropbox/Tanmay Singh/M61/data/rays_and_recipes_sid398784_snap099/debug_yt_alpha109_flip_OffAxisProjection_H_p0_number_density.png


[OK] wrote /Users/wavefunction/ASU Dropbox/Tanmay Singh/M61/data/rays_and_recipes_sid398784_snap099/debug_manual_alpha109_flip.png
[OK] wrote /Users/wavefunction/ASU Dropbox/Tanmay Singh/M61/data/rays_and_recipes_sid398784_snap099/debug_yt_alpha109_flip_OffAxisProjection_H_p0_number_density.png
[OK] wrote /Users/wavefunction/ASU Dropbox/Tanmay Singh/M61/data/rays_and_recipes_sid398784_snap099/compare_alpha109_flip.png
[DONE] /Users/wavefunction/ASU Dropbox/Tanmay Singh/M61/data/rays_and_recipes_sid398784_snap099
